In [1]:
import pandas as pd

In [13]:
name = "./old_train_results/Exp_result-vector_3000_bow-2023-09-17_19-cv=3.log"
bow_cv3 = pd.read_csv(name, delimiter="\s+")

In [16]:
bow_cv3

,fit_time,score_time,test_accuracy,train_accuracy,test_f1_score,train_f1_score,test_precision,train_precision,test_recall,train_recall
LogisticRegression,0.967305,0.026104,0.848333,1.000000,0.849492,1.000000,0.843886,1.000000,0.855333,1.000000
RidgeClassifier,2.402006,0.039038,0.800000,1.000000,0.806123,1.000000,0.782522,1.000000,0.831333,1.000000
SVC,16.532197,11.951077,0.805667,0.995833,0.787502,0.995834,0.867247,0.995339,0.721333,0.996333
SGDClassifier,0.702013,0.020524,0.746000,0.994500,0.672499,0.994452,0.945175,1.000000,0.524000,0.989000
Perceptron,0.498460,0.017421,0.815000,1.000000,0.820098,1.000000,0.798121,1.000000,0.843333,1.000000
GaussianNB,0.364445,0.102181,0.687667,0.918667,0.672530,0.924570,0.707017,0.862530,0.641333,0.996333
DecisionTreeClassifier,1.432976,0.030802,0.726000,0.845500,0.722360,0.842904,0.732447,0.856883,0.714667,0.830000
BaggingClassifier,23.211518,0.891012,0.784333,0.997500,0.773986,0.997496,0.813035,0.998997,0.739333,0.996000
AdaBoostClassifier,18.166416,0.669578,0.813000,0.867500,0.815648,0.868972,0.803735,0.859498,0.828000,0.878667
RandomForestClassifier,2.163243,0.060399,0.812000,0.882333,0.797831,0.874779,0.861340,0.934800,0.743333,0.822000


In [17]:
bow_cv3['train_accuracy'] / bow_cv3['test_accuracy']

LogisticRegression            1.178782
RidgeClassifier               1.250000
SVC                           1.236035
SGDClassifier                 1.333110
Perceptron                    1.226994
GaussianNB                    1.335918
DecisionTreeClassifier        1.164601
BaggingClassifier             1.271781
AdaBoostClassifier            1.067036
RandomForestClassifier        1.086617
GradientBoostingClassifier    1.187996
KNeighborsClassifier          1.007656
MLPClassifier                 1.194268
dtype: float64